In [1]:
import numpy as np
import pandas as pd

# Physical

## Sanitized Dataset

In [ ]:
def crate_sanitized_new_physical_stores_data(path):    
    # df_old= pd.read_excel('data/raw/20240509_Historicos_2da_Fase_Discontinuos.xlsx',sheet_name='Venta Física')
    # df_old = df_old.drop(0)
    
    df_new = pd.read_excel(path)
    df_new = df_new[df_new['type_store'] !=  'DIGITAL']
    # df_new = df_new.drop(['type_store'],axis=1)
    
    # Add the new data from Monday
    df_old = pd.DataFrame()
    # df_new = df_new.drop(0)
    
    df = pd.concat([df_old,df_new], ignore_index=True)
     
    df['SKU Credit Price including VAT ( IVA)'] = df['Sale Pesos Credit tax']
    df['SKU Cash Price including VAT ( IVA)']  = df['Cash Sale Pesos tax']
    df['Pilot/Control'] = df['type_store']
    df.drop(['Sale Pesos Credit tax','Cash Sale Pesos tax','type_store'], axis=1, inplace=True)

    ### 1. Sum of Cash Unit Sale and Sale of Credit Units cannot be 0
    row_sums = df['Sale of Credit Units'] + df['Cash Unit Sale']
    check_sums = row_sums != 0
    df = df[check_sums]

    ### 1. SKU Credit Price including VAT ( IVA) and Price To Elektra Credit (A) is same
    # check_sums = df['SKU Credit Price including VAT ( IVA)'] == df['Price To Elektra Credit (A)'] 
    # df = df[check_sums]

    ### 3. 'SKU Cash Price including VAT ( IVA)' and 'Cash Price To Elektra (A)' is same
    # check_sums = df['SKU Cash Price including VAT ( IVA)'] == df['Cash Price To Elektra (A)'] 
    # df = df[check_sums]
    # df.to_csv('../data/discontinuous/sanitized_discontinuous_current_stores_data.csv')
    return df

In [ ]:
old_master = pd.read_csv('data/Sanitized_Master_Physical_stores_21072024.csv')
df_new = crate_sanitized_new_physical_stores_data(path='data/raw/Ventas_25072024_Jueves.xlsx')
df_new = df_new[df_new['Day of Date']>='2024-07-22']
df_new
combined2 = pd.concat([old_master,df_new], ignore_index=True)
combined2.to_csv('data/Sanitized_Master_Physical_stores_24072024.csv')

## Scaled Dataset

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np

import os
def scale_prices(df, date_column, price_column, new_column_name='Scaled Price'):

    df = df.sort_values(by=date_column).reset_index(drop=True)
    print(len(df))
    first_price = df.loc[0, price_column]
    scaling_factor = 1000 / first_price
    
    # print('merged',df.columns)
    # print('Elektra Cash Price (De)',df['Elektra Cash Price (De)'])
    
    df['Scaled_Price_Credit_A'] = df[price_column] * scaling_factor
    df['Scaled_Price_Cash_A'] = df['Cash Price To Elektra (A)'] * scaling_factor
    df['Scaled_Price_Cash_DE'] = df['Elektra Cash Price (De)'] * scaling_factor
    df['Scaled_Price_Credit_DE'] = df['Elektra Credit Price (De)'] * scaling_factor
    # print('scaling_factor',scaling_factor)
    # print(df[['Elektra Credit Price (De)']].iloc[0])
    # print(df[['Scaled_Price_Credit_DE']].iloc[0])

    return df
def create_csv_credit_group_physical(df, column_name, value):
    filtered = df[df[column_name] == value]
    filtered = filtered[filtered['Sale of Credit Units'] != 0]
    filtered = filtered[[column_name, 'Cost of Sale Credit','Day of Date', 'Sale of Credit Units', 'Price To Elektra Credit (A)', 'Gross margin%', 'Total to pay average Elektra unit (10% Down payment)', 'Coppel Digital Price', 'Coppel Digital Subscription',
                         'Elektra Digital Price', 'Elektra Digital subscription', 'Elektra Credit Price\n(De)','Pilot/Control']]
    
    filtered['Day of Date'] = filtered['Day of Date'].astype(str)
    filtered['Day of Date'] = filtered['Day of Date'].apply(lambda x: x + ' 00:00:00' if len(x.split()) == 1 else x)

    filtered['Day of Date'] = pd.to_datetime(filtered['Day of Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
    
    
    filtered['Day of Date'] = filtered['Day of Date'].dt.strftime('%Y-%m-%d').astype(str)

    def clean_comp_data(df, col_name):
        df[col_name] = df[col_name].replace(' ', '0')
        df[col_name] = df[col_name].astype(float)
        df[col_name] = df[col_name].replace(0.0, pd.NA)
        return df

    filtered = clean_comp_data(filtered, 'Coppel Digital Price')
    filtered = clean_comp_data(filtered, 'Elektra Digital Price')

    def mode(x):
        return x.mode().max() if not x.mode().empty else np.nan

    aggregated_data = filtered.groupby(['Day of Date','Pilot/Control']).agg({
        'SKU': 'first',
        # 'Model':'first',
        'Sale of Credit Units': 'sum',
        'Price To Elektra Credit (A)': mode,
        'Elektra Credit Price\n(De)': mode,
        'Gross margin%': mode,
        'Total to pay average Elektra unit (10% Down payment)': mode,
        'Coppel Digital Price': mode,
        'Elektra Digital Price': mode,
        'Cost of Sale Credit': 'mean',
        # 'Pilot/Control':mode,
        # 'Carrier': 'first',
        # 'Brand': 'first',
        # 'Model': 'first'
    }).reset_index()
    
    if value == 31051005:
        aggregated_data.to_csv('aggregated_data_31051005.csv')
        

    aggregated_data['Day of Date'] = pd.to_datetime(aggregated_data['Day of Date'])
    aggregated_data.sort_values(by="Day of Date", inplace=True)

    aggregated_data = aggregated_data.fillna({
        'Sale of Credit Units': 0
    })

    aggregated_data.fillna(method="ffill", inplace=True)
    aggregated_data.fillna(method="bfill", inplace=True)

    aggregated_data['mean_credit_sales_last_week_lag2'] = aggregated_data['Sale of Credit Units'].rolling(window=7).mean().shift(2)
    aggregated_data['mean_credit_sales_last_week_lag2'] = aggregated_data['mean_credit_sales_last_week_lag2'].fillna(0)

    aggregated_data['Month'] = aggregated_data['Day of Date'].dt.month
    aggregated_data['Year'] = aggregated_data['Day of Date'].dt.year
    aggregated_data['Week Number'] = aggregated_data['Day of Date'].dt.isocalendar().week

    aggregated_data['Week Number within Month'] = aggregated_data['Day of Date'].apply(lambda x: (x.day - 1) // 7 + 1)

    aggregated_data.sort_values(by='Day of Date', inplace=True)

    aggregated_data['Days Since Introduction'] = (aggregated_data['Day of Date'] - aggregated_data['Day of Date'].min()).dt.days + 1

    aggregated_data['Elektra Credit Price (De)'] = aggregated_data['Elektra Credit Price\n(De)']
    aggregated_data.drop(['Elektra Credit Price\n(De)'], axis=1, inplace=True)

    aggregated_data.to_csv(f'Temp_Dirs_physical/{column_name}_data/{column_name}_{value}_temp_credit.csv', index=False)


def create_csv_cash_grouped_physical(df, column_name, value):
    filtered = df[df[column_name] == value]
    filtered = filtered[filtered['Sale of Credit Units'] == 0]
    filtered = filtered[[column_name, 'Day of Date', 'Cash Unit Sale', 'Cash Price To Elektra (A)', 'Gross margin%', 'Coppel Digital Price',
                         'Cost of Sale Credit','Elektra Digital Price', 'Elektra Cash Price\n(De)','Pilot/Control']]
    # filtered['Day of Date'] = pd.to_datetime(filtered['Day of Date'])
    
    # Convert dates with time
    filtered['Day of Date'] = filtered['Day of Date'].astype(str)

    # Add ' 00:00:00' to values that contain only dates
    filtered['Day of Date'] = filtered['Day of Date'].apply(lambda x: x + ' 00:00:00' if len(x.split()) == 1 else x)

    # Convert the column to datetime
    filtered['Day of Date'] = pd.to_datetime(filtered['Day of Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
    filtered['Day of Date'] = filtered['Day of Date'].dt.strftime('%Y-%m-%d').astype(str)

    def clean_comp_data(df, col_name):
        df[col_name] = df[col_name].replace(' ', '0')
        df[col_name] = df[col_name].astype(float)
        df[col_name] = df[col_name].replace(0.0, pd.NA)
        return df

    filtered = clean_comp_data(filtered, 'Coppel Digital Price')
    filtered = clean_comp_data(filtered, 'Elektra Digital Price')

    def mode(x):
        return x.mode().max() if not x.mode().empty else np.nan
    
    # filtered.to_csv('3100.csv')
    aggregated_data = filtered.groupby(['Day of Date','Pilot/Control']).agg({
        'SKU': 'first',
        # 'Model':'first',
        'Cash Unit Sale': 'sum',
        'Cash Price To Elektra (A)': mode,
        'Elektra Cash Price\n(De)': mode,
        'Gross margin%': mode,
        'Coppel Digital Price': mode,
        'Elektra Digital Price': mode,
        # 'Carrier': 'first',
        'Cost of Sale Credit': 'mean',
        # 'Pilot/Control':mode,
        # 'Brand': 'first',
        # 'Model': 'first'
    }).reset_index()
    # aggregated_data.to_csv('3120.csv')

    aggregated_data['Day of Date'] = pd.to_datetime(aggregated_data['Day of Date'])
    aggregated_data.sort_values(by="Day of Date", inplace=True)

    aggregated_data.fillna({
        'Cash Unit Sale': 0
    }, inplace=True)

    aggregated_data.fillna(method="ffill", inplace=True)
    aggregated_data.fillna(method="bfill", inplace=True)

    aggregated_data['mean_cash_sales_last_week_lag2'] = aggregated_data['Cash Unit Sale'].rolling(window=7).mean().shift(2)
    aggregated_data['mean_cash_sales_last_week_lag2'] = aggregated_data['mean_cash_sales_last_week_lag2'].fillna(0)

    aggregated_data['Month'] = aggregated_data['Day of Date'].dt.month
    aggregated_data['Year'] = aggregated_data['Day of Date'].dt.year
    aggregated_data['Week Number'] = aggregated_data['Day of Date'].dt.isocalendar().week

    aggregated_data['Week Number within Month'] = aggregated_data['Day of Date'].apply(lambda x: (x.day - 1) // 7 + 1)

    aggregated_data.sort_values(by='Day of Date', inplace=True)

    aggregated_data['Days Since Introduction'] = (aggregated_data['Day of Date'] - aggregated_data['Day of Date'].min()).dt.days + 1

    aggregated_data['Elektra Cash Price (De)'] = aggregated_data['Elektra Cash Price\n(De)']
    aggregated_data.drop(['Elektra Cash Price\n(De)'], axis=1, inplace=True)
    
    aggregated_data.to_csv(f'Temp_Dirs_physical/{column_name}_data/{column_name}_{value}_temp_cash.csv', index=False)


def merge_df_grouped_physical(df, column_name, value):
    df_credit = pd.read_csv(f'Temp_Dirs_physical/{column_name}_data/{column_name}_{value}_temp_credit.csv')
    df_cash = pd.read_csv(f'Temp_Dirs_physical/{column_name}_data/{column_name}_{value}_temp_cash.csv')
    
    df_cash = df_cash[['Day of Date','Pilot/Control', 'Cash Unit Sale', 'Cash Price To Elektra (A)', 'Elektra Cash Price (De)']]
    
    merged = pd.merge(df_credit, df_cash, on=['Day of Date','Pilot/Control'], how='outer')
    
    # print('merged',merged.info())
    
    merged.fillna({
        'Cash Unit Sale': 0,
        'Sale of Credit Units': 0
    }, inplace=True)
    
    merged.fillna(method="ffill", inplace=True)
    merged.fillna(method="bfill", inplace=True)
        
    if not merged.empty:
        merged = scale_prices(merged, 'Day of Date', 'Price To Elektra Credit (A)')

    

    merged.fillna(method="ffill", inplace=True)
    merged.fillna(method="bfill", inplace=True)
    
    # merged = df_credit

    merged.to_csv(f'Temp_Dirs_physical/{column_name}_data_merged/{column_name}_{value}_temp.csv', index=False)

def combine_csv_files(directory_path, output_file):
    csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]
    
    df_list = []

    for csv_file in csv_files:
        file_path = os.path.join(directory_path, csv_file)
        df = pd.read_csv(file_path)
        df['Day of Date'] = pd.to_datetime(df['Day of Date'])
        df_list.append(df)
    
    combined_df = pd.concat(df_list, ignore_index=True)

    combined_df['Day of Date'] = pd.to_datetime(combined_df['Day of Date'])
    combined_df.sort_values(by='Day of Date', inplace=True)

    combined_df.to_csv(output_file, index=False)





In [ ]:

physical_ls = [31057271, 31057273, 31057286, 31057163, 31057164, 31057172,
       31055497, 31055496, 31058953, 31058928, 31055495, 31057162,
       31057150, 31057161, 31056539, 31056541, 31056540, 31058721,
       31053498, 31053497, 31058051, 31058056, 31059004, 31058997,
       31057958, 31057948, 31057959, 31057947, 31058023, 31058018,
       31057257, 31057258, 31057263, 31059186, 31059202, 31057715,
       31057707, 31058923, 31058932, 31058942, 31058922, 31057048,
       31057047, 31058674, 31058675, 31058672, 31058673, 31058676,
       31058639, 31058640, 31058643, 31058652, 31058642, 31058659,
       31057523, 31058048, 31058054, 31055524, 31055523, 31058887,
       31058869, 31058008, 31058000, 31058005, 31051762, 31059222,
       31058905, 31058894, 31058030, 31058047, 31057579, 31057592,
       31057589, 31057590, 31057580, 31057561, 31057581, 31057519,
       31057661, 31057680, 31057666, 31058082, 31058078, 31058071,
       31057343, 31057337, 31057311, 31057400, 31057384, 31058847,
       31058968, 31058984, 31057087, 31057088, 31058993, 31058979,
       31058988, 31056545, 31056544, 31055343, 31058034, 31058028,
       31059240, 31059263, 31058767, 31058766, 31058087, 31058094,
       31058971, 31058992, 31057241, 31057247, 31058989, 31058986,
       31057239, 31057245, 31059073, 31059068, 31057246, 31057240,
       31057236, 31057249, 31059119, 31059074, 31058033, 31058027,
       31059260, 31058710, 31058754, 31058032, 31058025, 31059261,
       31059241, 31058896, 31058723, 31057753, 31057743, 31058022,
       31059139, 31059203, 31057767, 31057776, 31058977, 31057732,
       31055552, 31055554, 31055383, 31045304, 31050860, 31055362,
       31047746, 31055363, 31032647, 31052376, 31055352, 31052384,
       31055310, 31057260, 31057283, 31050784, 31050785, 31051900,
       31051907, 31051954, 31051955, 31051906, 31051965, 31051962,
       31051898, 31054964, 31054965, 31055603, 31055604, 31052267,
       31049642, 31050805, 31055373, 31055374, 31057800, 31057807,
       31057811, 31051491, 31051512, 31057272, 31057285, 31050797,
       31050798, 31048023, 31050806, 31048290, 31051504, 31051505,
       31041272, 31053174, 31053175, 31045254, 31045268, 31056551,
       31056552, 31056553, 31023696, 31023697, 31044891, 31047401,
       31050116, 31047968, 31047969, 31044050, 31048641, 31055521,
       31055526, 31055692, 31055702, 31045301, 31054264, 31054272,
       31056520, 31056530, 31055129, 31052285, 31057217, 31057223,
       31044759, 31051425, 31051409, 31051415, 31051929, 31051937,
       31050449, 31055509, 31055510, 31055530, 31055531, 31055542,
       31051024, 31057153, 31057154, 31057155, 31057861, 31057871,
       31057872, 31047852, 31047853, 31047854, 31050996, 31050998,
       31044773, 31044774, 31051032, 31051033, 31051034, 31051035,
       31044766, 31051002, 31051004, 31051005, 31058937, 31058951,
       31058952, 31051597, 31051599, 31051600, 31050458, 31057598,
       31057607, 31051973, 31051974, 31051941, 31051982, 31055766,
       31055768, 31057908, 31057912, 31055765, 31055767, 31057801,
       31057809, 31050186, 31051148, 31055200, 31052400, 31054970,
       31052120, 31052121, 31057256, 31057270, 31051165, 31051038,
       31057259, 31057264, 31047788, 31043249, 31050114, 31050115,
       31050108, 31050109, 31050111, 31050112, 31057252, 31047888,
       31047886, 31057267, 31051481, 31051499, 31051501, 31057237,
       31057250, 31057765, 31052027, 31052028, 31051547, 31055644,
       31055645, 31051424, 31049136, 31051523, 31051524, 31055599,
       31055600, 31050260, 31050261, 31051093, 31051094, 31051434,
       31051435, 31048066, 31053162, 31053163, 31048650, 31055757,
       31055758, 31044904, 31047799, 31047800, 31055314, 31055315,
       31055029, 31052398, 31052399, 31056536, 31056537, 31057637,
       31057594, 31057603, 31053525, 31053526, 31054866, 31054867,
       31054868, 31047890, 31051654, 31051655, 31051656, 31051694,
       31051695, 31055563, 31055564, 31055565, 31059108, 31059122,
       31059109, 31059120, 31057248, 31057262, 31045265, 31057238,
       31057244, 31052116, 31052118, 31057745, 31059146, 31055731,
       31055748, 31057742, 31057752, 31055749, 31055742, 31055750,
       31030392, 31056563, 31056564, 31055735, 31055752, 31055745,
       31057734, 31057711, 31057725, 31057735, 31057909, 31057913,
       31057916, 31057953, 31057768, 31057777, 31049532, 31049533,
       31048227, 31036667, 31050594, 31050595, 31056360, 31057714,
       31055490, 31057691, 31057695, 31058868, 31058879, 31055416,
       31055417, 31055630, 31055631, 31052240, 31052241, 31051678,
       31051542, 31051543, 31055194, 31048002, 31048298, 31048299,
       31050521, 31050525, 31050534, 31050536, 31053203, 31053204,
       31050968, 31050971, 31050976, 31050977, 31047427, 31048389,
       31055895, 31055896, 31055016, 31055017, 31055108, 31055109,
       31058009, 31058015, 31052438, 31052440, 31057018, 31057019,
       31047340, 31051356, 31051357, 31048865, 31048866, 31056272,
       31056273, 31056274, 31056635, 31056636, 31056637, 31056262,
       31057206, 31057212, 31057213, 31057613, 31057625, 31057632,
       31051838, 31051839, 31051840, 31052155, 31052156, 31045043,
       31050179, 31057652, 31057737, 31052082, 31052083, 31047831,
       31052091, 31052112, 31056959, 31056960, 31057899, 31057907,
       31048828, 31048830, 31057027, 31057028, 31050297, 31055158,
       31055159, 31055160, 31051402, 31053437, 31053438, 31053439,
       31057356, 31045187, 31059053, 31050323, 31056406, 31050360,
       31057403, 31057419, 31057425, 31058720, 31057432, 31057447,
       31054895, 31054896, 31054897, 31054923, 31057441, 31057463,
       31057856, 31057870]

models = ['A24', 'A54,  + 128 GB', 'A04 64GB,', 'GALAXY A04 64GB', 'A34,',
       'A04E 64GB,', 'G42,', 'G14 128GB', 'A58', 'RENO 10 5G', 'A78',
       'XIOAMI  12', 'REDMI 12', 'NOTE 12  4+128', '13T BUNDLE',
       'MAGIC 5 LITE', '90 LITE', '90 256GB,', 'X7A,', '15 PLUS 512 GB,',
       '15 512 GB,', '15 256 GB,', '15 PRO 1 TB,', '14 PLUS 128GB', 'Y36',
       'V25,', 'TELCEL  A58 R9,', 'TELCEL  A78 R9,', 'TELCEL  A78 R7,',
       'TELCEL  11 R9,', 'TELCEL  G84 BUND R9,', 'TELCEL  G84 5G R9,',
       'TELCEL  G84 5G R7,', 'TELCEL  G14 R9,', 'TELCEL  A34 R9,',
       'TELCEL  A54 128 R9,', 'TELCEL  A24 R9,', 'TELCEL  Y36 128G R9,',
       'TELCEL  Y36 128G R8,', 'TELCEL  NOTE 12S R9,',
       'TELCEL  NOTE 12 R9,', 'TELCEL  REDMI A2 R9,',
       'TELCEL  MAGIC 5L R9,', 'TELCEL  90 LITE R9,', 'TELCEL  X7A R9,',
       'AT&T  90 LITE,', 'AT&T  90,', 'AT&T  X7A,', 'AT&T  X6,',
       'AT&T  RENO 10,', 'AT&T  RENO 10 BUNDLE,', 'AT&T  A58,',
       'AT&T  A78,', 'AT&T  A54 128GB,', 'AT&T  A14 4G,',
       'AT&T  A34 128GB,', 'AT&T  A04 64GB', 'AT&T  G54,', 'ATT  G13',
       'AT&T  G13,', 'AT&T  NOTE 12,', 'ATT  NOTE 12,',
       'AT&T  REDMI A2 64 GB', 'MOVISTAR  A78,', 'MOVISTAR  A78 CB,',
       'MOVISTAR  A58,', 'MOVISTAR  RENO 10,', 'MOVISTAR  RENO 10 CB',
       'MOVISTAR  G54,', 'MOVISTAR  G13,', 'MOVISTAR  G14',
       'MOVISTAR  G14,', 'MOVISTAR  EDGE 40', 'MOVISTAR  NOTE 12,',
       'MOVISTAR  A24,', 'MOVISTAR  A04 64,', 'APPLE  12 (64GB),',
       '14 PLUS 128 GB  MEDIANO', 'BLU G61,', 'HISENSE E20S,',
       'HISENSE E20S', 'HISENSE E50,', 'HISENSE E60 LITE', 'HISENSE H11,',
       'HISENSE V40S', 'HISENSE V60', 'X6,', 'X8A+ EARBUDS', 'HUAWEI Y9A',
       'LANIX ALPHA 5V,', 'LANIX ALPHA 9V, ACERO', 'LANIX ILIUM M7V,',
       'LANIX ILIUM M9V,', 'LANIX X560,', 'LANIX X770,', 'LANIX X860,',
       'E20,', 'E22 64GB,', 'E32,', 'E7I POWER + REGALO,',
       'EDGE 20 LITE,', 'EDGE 30,', 'EDGE 40  ECLIPSE', 'EDGE 40  NEBULA',
       'EDGE 40', 'G22,', 'G23', 'G31,  METEORO', 'G31,  CIELO', 'G50,',
       'G51,  OTO?O', 'G60S + AUDIFONOS, AQUA', 'G60S, AQUA', 'G60S,',
       'G8 POWER LITE TURQUESA', 'G82 5G,', 'G60, .', 'G72,',
       'XT1543 MOTO G,', 'NUBIA N40,', 'NUBIA N41,', 'NUBIA N61,',
       'ONE PLUS N10, MIDNIGHT ICE', 'A16,', 'A16  DOBLE SIM', 'A17',
       'A54,', 'A57', 'A57  4 + 128', 'A77', 'RENO 7', 'RENO 7 (256)',
       'REALME 7,', 'REALME 8I,', 'REALME C21Y,', 'REALME C35 4 + 64',
       'A03 (128+4),', 'A04 32GB,', 'A04S,', 'A13,', 'A14,',
       'GALAXY A14,', 'A22,', 'A23,', 'A32,', 'A33,', 'A52,', 'A53,',
       'GALAXY A54 256GB', 'S20 FE 5G 128GB,', 'S21 FE 256+8,',
       'SMARTPHONE V25E COLOR', 'SMARTPHONE V25E COLOR VIO', 'Y01,',
       'Y15S,', 'Y22S,',  'Y35,', 'NOTE 12S BUNDLE',
       'NOTE 10 5G,', 'REDMI NOTE 11 PRO 5G,', 'REDMI NOTE 11PRO 5G BUN,',
       'REDMI 10 2022', 'REDMI 10A,', 'REDMI 10C,', '12C  4+128',
       'REDMI NOTE 11,  MARINO', 'REDMI NOTE 11S,',
       'NOTE 12 PRO+ 5G  8+256', 'REDMI NOTE 10S,', 'ZTE BLADE A5 2020,',
       'ZTE BLADE A71,', 'ZTE BLADE A31,', 'ZTE BLADE A51,',
       'AT&T  12 64GB,', 'AT&T HISENSE H50 LITE,', 'AT&T HISENSE U50,',
       'ATT  X5', 'AT&T  X7,', 'AT&T  X8,', 'ATT  X8A BUNDLE',
       'AT&T  X8A,', 'AT&T  X9,', 'AT&T HUAWEI NOVA 9SE,', 'AT&T  E22,',
       'AT&T  EDGE 20,', 'AT&T  EDGE 20 LITE,', 'AT&T  G22,',
       'AT&T  G32,', 'AT&T  G41,  ONIX', 'AT&T  G41,  AMANECER',
       'AT&T  G51', 'AT&T  G60', 'AT&T  G60S', 'AT&T  G82 5G',
       'AT&T  A16,', 'AT&T  A17,', 'AT&T  A53.,', 'AT&T  A54,',
       'AT&T  A57,', 'AT&T  A77,', 'AT&T  RENO 7,  AURORA',
       'AT&T  RENO 7,  COSMICO', 'AT&T  RENO 7 256GB,',
       'AT&T  A04 128GB,', 'AT&T  A04E 32GB,', 'AT&T  REDMI 10A,',
       'AT&T  REDMI 10C,', 'AT&T  REDMI NOTE 10S,  OCEANO',
       'AT&T  REDMI NOTE 11,  ESTELAR', 'AT&T  REDMI NOTE 11,  LUNAR',
       'AT&T  REDMI NOTE 11,  NOCTURNO', 'AT&T  NOTE 11S,',
       'AT&T  NOTE 11S,  NOCTURNO', 'AT&T  REDMI 11S BUNDLE,',
       'AT&T  NOTE 12 BUND,', 'AT&T  NOTE 12S BDL, NEGR',
       'AT&T  NOTE 12S BDL,', 'ATT  REDMI 12C,', 'AT&T  REDMI 9A,',
       'ATT  REDMI A2', 'AT&T  REDMI NOTE 11 PRO 5G,  C',
       'AT&T  REDMI NOTE 11 PRO 5G,  L', 'MOVISTAR  X5 SD,',
       'MOVISTAR  X5,', 'MOVISTAR  E20', 'MOVISTAR  E22,',
       'MOVISTAR  E32', 'MOVISTAR  G22', 'MOVISTAR  G32',
       'MOVISTAR NOKIA 5,', 'MOVISTAR  A17,', 'MOVISTAR  A57',
       'MOVISTAR  A04 128GB', 'MOVISTAR  A04E, COBRE', 'MOVISTAR  A04S,',
       'MOVISTAR  A14,', 'MOVISTAR  NOTE 12 CB,',
       'MOVISTAR XIOAMI NOTE 12 CB,', 'MOVI  NOTE 12S CB,',
       'MOVISTAR  REDMI 12C,', 'MOVISTAR  REDMI A2 32GB,',
       'MOVISTAR  REDMI A2 32GB, VERD', 'TELCEL ALCATEL 1 R9,',
       'TELCEL ALCATEL 1B R9,', 'TELCEL ALCATEL 5024A 1S R9,',
       'TELCEL ALCATEL 1SE 64GB R9,', 'TELCEL  12 R9,',
       'TELCEL  MAGIC 5L BU R9 PL', 'TELCEL  X6 R9,',
       'TELCEL  X8A 5G R9,', 'TELCEL  X8A 4G R9,', 'TELCEL  E20',
       'TELCEL  E22 R9,', 'TELCEL  E32 R9,', 'TELCEL  EDGE 20 LITE R9,',
       'TELCEL  G22 R9,', 'TELCEL  G32 R9,', 'TELCEL  G50 R9,',
       'TELCEL  G60S R9,', 'TELCEL  G71 5G R5,', 'TELCEL  G71 5G R6,',
       'TELCEL  G71 5G R9,', 'TELCEL  G82 5G R9,',
       'TELCEL  G60 128GB R6,', 'TELCEL  G60 128GB R9,',
       'TELCEL ONEPLUS N100  R9', 'TELCEL  A16 R9,', 'TELCEL  A17 R9,',
       'TELCEL  A57 R9,', 'TELCEL  A77 R9,', 'TELCEL  RENO 10 R9,',
       'TELCEL  RENO 7 R9,', 'TELCEL  RENO 7 256GB R9,',
       'TELCEL REALME C15 R9,', 'TELCEL  A03 32GB R9,',
       'TELCEL  A03S R9,', 'TELCEL  A04 R9,', 'TELCEL  A04E 32GB R9,',
       'TELCEL  A04E 32GB R9, COBRE', 'TELCEL  A04S R9,',
       'TELCEL  A14 R9,', 'TELCEL  A14 4G R9, NEGR', 'TELCEL  A14 4G R9,',
       'TELCEL  A14 4G R9, VERD', 'TELCEL  A22 5G R9,', 'TELCEL  A33 R9,',
       'TELCEL  A52 R9.,', 'TELCEL TCL 10L 256GB  R9', 'TELCEL  V25E R9,',
       'TELCEL  Y01 R9,', 'TELCEL  Y11S R9,', 'TELCEL  Y15S R9,',
       'TELCEL  Y22S R9,', 'TELCEL  Y22S 4RAM R9,', 'TELCEL  Y33 R9,',
       'TELCEL  Y35 R9,', 'TELCEL WIKO T3 R9,', 'TELCEL  REDMI A1 R9,',
       'TELCEL  NOTE 11S R9,', 'TELCEL  REDMI 10A R9,',
       'TELCEL  REDMI 12C R9,', 'TELCEL  REDMI 9T R9.,',
       'TELCEL ZTE BLADE A33S R9, VERD', 'TELCEL ZTE BLADE A51+AUD R9,',
       'TELCEL ZTE A52 LITE R9,', 'TELCEL ZTE BLADE A71+SMARTW R9,',
       'TELCEL ZTE BLADE A53 R9,', 'TELCEL ZTE A53 64GB R9,',
       'TELCEL ZTE BLADE L220 R9,', 'TELCEL ZTE V40 SMART R9,',
       'TELCEL ZTE V40 VITA R9,', 'TELCEL ZTE BLADE V41 SMART R9,',
       'TELCEL ZTE V41 SMAR 64+4 R9 AZ', 'TELCEL ZTE V41 SMAR 64+4 R9 GR']

digital_df = pd.read_csv('data/Sanitized_Master_Physical_stores_24072024.csv')

sku_map = pd.read_csv('data/helper/sku_product_name_map.csv')

digital_df = pd.merge(digital_df, sku_map, how='left',on='SKU')
# print(physical_ls)
for sku in physical_ls:
    print(sku)
    create_csv_cash_grouped_physical(digital_df,'SKU',sku)
    create_csv_credit_group_physical(digital_df,'SKU',sku)
    merge_df_grouped_physical(digital_df,'SKU',sku)
    # break

directory_path = 'Temp_Dirs_physical/SKU_data_merged/'
output_file = 'data/scaled_Sanitized_Master_Physical_stores_24072024.csv'
combine_csv_files(directory_path, output_file)

# Digital

## Sanitized Dataset

In [ ]:
def create_sanitized_digital_new_stores_data(path):    
    # df_old = pd.read_excel('data/raw/20240515_Historicos_2da_Fase.xlsx',sheet_name='Digital Sales')
    # df_old = df_old.drop(0)
    
    # Add the new data from Monday
    df_old = pd.DataFrame()
    df_new = pd.read_excel(path)
    df_new = df_new[df_new['type_store'] ==  'DIGITAL']
    # df_new = df_new.drop(['type_store'],axis=1)
    # df_new = df_new.drop(0)
    
    df = pd.concat([df_old,df_new], ignore_index=True)
    
    df['SKU Credit Price including VAT ( IVA)'] = df['Sale Pesos Credit tax']
    df['SKU Cash Price including VAT ( IVA)']  = df['Cash Sale Pesos tax']
    df['Pilot/Control'] = df['type_store']
    df.drop(['Cash Sale Pesos tax','Sale Pesos Credit tax','type_store'], axis=1, inplace=True)

    ### 1. Sum of Cash Unit Sale and Sale of Credit Units cannot be 0
    row_sums = df['Sale of Credit Units'] + df['Cash Unit Sale']
    check_sums = (row_sums != 0) & (row_sums != -1)
    df = df[check_sums]
   

    ### 1. SKU Credit Price including VAT ( IVA) and Price To Elektra Credit (A) is same
    # check_sums = df['SKU Credit Price including VAT ( IVA)'] == df['Price To Elektra Credit (A)'] 
    # df = df[check_sums]
    

    ### 3. 'SKU Cash Price including VAT ( IVA)' and 'Cash Price To Elektra (A)' is same
    # check_sums = df['SKU Cash Price including VAT ( IVA)'] == df['Cash Price To Elektra (A)'] 
    # df = df[check_sums]
    # df.to_csv('dele.csv')
    
    # df.to_csv('../data/digital/sanitized_digital_current_stores_data.csv')

    return df

In [ ]:
old_master = pd.read_csv('data/Sanitized_Master_Digital_stores_07072024.csv')
df_new = create_sanitized_digital_new_stores_data(path='data/raw/Ventas_22072024_Lunes.xlsx')
df_new['Day of Date'] = pd.to_datetime(df_new['Day of Date'])

df_new = df_new[df_new['Day of Date']>='2024-07-08']

combined2 = pd.concat([old_master,df_new], ignore_index=True)
combined2.to_csv('data/Sanitized_Master_Digital_stores_21072024.csv')

## Scaled Dataset

In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np


def create_csv_credit_group_digital(df, column_name, value):
    filtered = df[df[column_name] == value]
    filtered = filtered[filtered['Sale of Credit Units'] != 0]
    filtered = filtered[['SKU', 'Cost of Sale Credit','Day of Date', 'Sale of Credit Units', 'Price To Elektra Credit (A)', 'Gross margin%', 'Total to pay average Elektra unit (10% Down payment)', 'Coppel Digital Price', 'Coppel Digital Subscription',
                         'Elektra Digital Price', 'Elektra Digital subscription', 'Elektra Credit Price\n(De)','Pilot/Control']]
    filtered['Day of Date'] = pd.to_datetime(filtered['Day of Date'])
    
    # filtered['Day of Date'] = filtered['Day of Date'].astype(str)
    # filtered['Day of Date'] = filtered['Day of Date'].apply(lambda x: x + ' 00:00:00' if len(x.split()) == 1 else x)

    # filtered['Day of Date'] = pd.to_datetime(filtered['Day of Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
    
    # filtered['Day of Date'] = filtered['Day of Date'].dt.strftime('%Y-%m-%d').astype(str)
    def clean_comp_data(df, col_name):
        df[col_name] = df[col_name].replace(' ', '0')
        df[col_name] = df[col_name].astype(float)
        df[col_name] = df[col_name].replace(0.0, pd.NA)
        return df

    filtered = clean_comp_data(filtered, 'Coppel Digital Price')
    filtered = clean_comp_data(filtered, 'Elektra Digital Price')

    def mode(x):
        return x.mode().max() if not x.mode().empty else np.nan

    aggregated_data = filtered.groupby(['Day of Date']).agg({
        'SKU': 'first',
        'Sale of Credit Units': 'sum',
        'Price To Elektra Credit (A)': mode,
        'Elektra Credit Price\n(De)': mode,
        'Gross margin%': mode,
        'Total to pay average Elektra unit (10% Down payment)': mode,
        'Coppel Digital Price': mode,
        'Elektra Digital Price': mode,
        'Cost of Sale Credit': 'mean',
        # 'Pilot/Control':mode,
        # 'Carrier': 'first',
        # 'Brand': 'first',
        # 'Model': 'first'
    }).reset_index()

    aggregated_data['Day of Date'] = pd.to_datetime(aggregated_data['Day of Date'])
    aggregated_data.sort_values(by="Day of Date", inplace=True)

    aggregated_data.fillna({
        'Sale of Credit Units': 0
    }, inplace=True)

    aggregated_data.fillna(method="ffill", inplace=True)
    aggregated_data.fillna(method="bfill", inplace=True)

    aggregated_data['mean_credit_sales_last_week_lag2'] = aggregated_data['Sale of Credit Units'].rolling(window=7).mean().shift(2)
    aggregated_data['mean_credit_sales_last_week_lag2'] = aggregated_data['mean_credit_sales_last_week_lag2'].fillna(0)

    aggregated_data['Month'] = aggregated_data['Day of Date'].dt.month
    aggregated_data['Year'] = aggregated_data['Day of Date'].dt.year
    aggregated_data['Week Number'] = aggregated_data['Day of Date'].dt.isocalendar().week

    aggregated_data['Week Number within Month'] = aggregated_data['Day of Date'].apply(lambda x: (x.day - 1) // 7 + 1)

    aggregated_data.sort_values(by='Day of Date', inplace=True)

    aggregated_data['Days Since Introduction'] = (aggregated_data['Day of Date'] - aggregated_data['Day of Date'].min()).dt.days + 1

    aggregated_data['Elektra Credit Price (De)'] = aggregated_data['Elektra Credit Price\n(De)']
    aggregated_data.drop(['Elektra Credit Price\n(De)'], axis=1, inplace=True)

    aggregated_data.to_csv(f'Temp_Dirs_Digital/{column_name}_data/{column_name}_{value}_temp_credit.csv', index=False)


def create_csv_cash_grouped_digital(df, column_name, value):
    filtered = df[df[column_name] == value]
    filtered = filtered[filtered['Sale of Credit Units'] == 0]
    filtered = filtered[['SKU', 'Day of Date', 'Cash Unit Sale', 'Cash Price To Elektra (A)', 'Gross margin%', 'Coppel Digital Price',
                         'Cost of Sale Credit','Elektra Digital Price', 'Elektra Cash Price\n(De)','Pilot/Control']]
    filtered['Day of Date'] = pd.to_datetime(filtered['Day of Date'])
    
    # Convert dates with time
    # filtered['Day of Date'] = filtered['Day of Date'].astype(str)

    # Add ' 00:00:00' to values that contain only dates
    # filtered['Day of Date'] = filtered['Day of Date'].apply(lambda x: x + ' 00:00:00' if len(x.split()) == 1 else x)

    # Convert the column to datetime
    # filtered['Day of Date'] = pd.to_datetime(filtered['Day of Date'], errors='coerce', format='%Y-%m-%d %H:%M:%S')
    # filtered['Day of Date'] = filtered['Day of Date'].dt.strftime('%Y-%m-%d').astype(str)

    def clean_comp_data(df, col_name):
        df[col_name] = df[col_name].replace(' ', '0')
        df[col_name] = df[col_name].astype(float)
        df[col_name] = df[col_name].replace(0.0, pd.NA)
        return df

    filtered = clean_comp_data(filtered, 'Coppel Digital Price')
    filtered = clean_comp_data(filtered, 'Elektra Digital Price')

    def mode(x):
        return x.mode().max() if not x.mode().empty else np.nan
    
    filtered.to_csv('3100.csv')
    aggregated_data = filtered.groupby(['Day of Date']).agg({
        'SKU': 'first',
        'Cash Unit Sale': 'sum',
        'Cash Price To Elektra (A)': mode,
        'Elektra Cash Price\n(De)': mode,
        'Gross margin%': mode,
        'Coppel Digital Price': mode,
        'Elektra Digital Price': mode,
        # 'Carrier': 'first',
        'Cost of Sale Credit': 'mean',
        # 'Pilot/Control':mode,
        # 'Brand': 'first',
        # 'Model': 'first'
    }).reset_index()
    aggregated_data.to_csv('3120.csv')

    aggregated_data['Day of Date'] = pd.to_datetime(aggregated_data['Day of Date'])
    aggregated_data.sort_values(by="Day of Date", inplace=True)

    aggregated_data.fillna({
        'Cash Unit Sale': 0
    }, inplace=True)

    aggregated_data.fillna(method="ffill", inplace=True)
    aggregated_data.fillna(method="bfill", inplace=True)

    aggregated_data['mean_cash_sales_last_week_lag2'] = aggregated_data['Cash Unit Sale'].rolling(window=7).mean().shift(2)
    aggregated_data['mean_cash_sales_last_week_lag2'] = aggregated_data['mean_cash_sales_last_week_lag2'].fillna(0)

    aggregated_data['Month'] = aggregated_data['Day of Date'].dt.month
    aggregated_data['Year'] = aggregated_data['Day of Date'].dt.year
    aggregated_data['Week Number'] = aggregated_data['Day of Date'].dt.isocalendar().week

    aggregated_data['Week Number within Month'] = aggregated_data['Day of Date'].apply(lambda x: (x.day - 1) // 7 + 1)

    aggregated_data.sort_values(by='Day of Date', inplace=True)

    aggregated_data['Days Since Introduction'] = (aggregated_data['Day of Date'] - aggregated_data['Day of Date'].min()).dt.days + 1

    aggregated_data['Elektra Cash Price (De)'] = aggregated_data['Elektra Cash Price\n(De)']
    aggregated_data.drop(['Elektra Cash Price\n(De)'], axis=1, inplace=True)
    
    aggregated_data.to_csv(f'Temp_Dirs_Digital/{column_name}_data/{column_name}_{value}_temp_cash.csv', index=False)



In [ ]:
import os
def scale_prices(df, date_column, price_column, new_column_name='Scaled Price'):

    df = df.sort_values(by=date_column).reset_index(drop=True)
    print(len(df))
    first_price = df.loc[0, price_column]
    scaling_factor = 1000 / first_price
    
    df[ 'Scaled_Price_Credit_A'] = df[price_column] * scaling_factor
    df[ 'Scaled_Price_Cash_A'] = df['Cash Price To Elektra (A)'] * scaling_factor
    df[ 'Scaled_Price_Cash_DE'] = df['Elektra Cash Price (De)'] * scaling_factor
    df[ 'Scaled_Price_Credit_DE'] = df['Elektra Credit Price (De)'] * scaling_factor

    return df

def merge_df_grouped_digital(df, column_name, value):
    df_credit = pd.read_csv(f'Temp_Dirs_Digital/{column_name}_data/{column_name}_{value}_temp_credit.csv')
    df_cash = pd.read_csv(f'Temp_Dirs_Digital/{column_name}_data/{column_name}_{value}_temp_cash.csv')
    
    df_cash = df_cash[['Day of Date', 'Cash Unit Sale', 'Cash Price To Elektra (A)', 'Elektra Cash Price (De)']]
    
    merged = pd.merge(df_credit, df_cash, on=['Day of Date'], how='left')
    
    if not merged.empty:
        merged = scale_prices(merged, 'Day of Date', 'Price To Elektra Credit (A)')

    merged.fillna({
        'Cash Unit Sale': 0,
        'Sale of Credit Units': 0
    }, inplace=True)

    merged.fillna(method="ffill", inplace=True)
    merged.fillna(method="bfill", inplace=True)

    merged.to_csv(f'Temp_Dirs_Digital/{column_name}_data_merged/{column_name}_{value}_temp.csv', index=False)

def combine_csv_files(directory_path, output_file):
    csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]
    
    df_list = []

    for csv_file in csv_files:
        file_path = os.path.join(directory_path, csv_file)
        df = pd.read_csv(file_path)
        df['Day of Date'] = pd.to_datetime(df['Day of Date'])
        df_list.append(df)
    
    combined_df = pd.concat(df_list, ignore_index=True)

    combined_df['Day of Date'] = pd.to_datetime(combined_df['Day of Date'])
    combined_df.sort_values(by='Day of Date', inplace=True)

    combined_df.to_csv(output_file, index=False)




In [ ]:

digital_df = pd.read_csv('data/Sanitized_Master_Digital_stores_12062024.csv')

for sku in physical_ls:
    print(sku)
    create_csv_cash_grouped_digital(digital_df,'SKU',sku)
    create_csv_credit_group_digital(digital_df,'SKU',sku)
    merge_df_grouped_digital(digital_df,'SKU',sku)

directory_path = 'Temp_Dirs_Digital/sku_data_merged/'
output_file = 'data/scaled_Sanitized_Master_Digital_stores_12062024.csv'
combine_csv_files(directory_path, output_file)